In [8]:
import pandas as pd
import pandas as pd
import snappy
import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings("ignore")

In [13]:
df_reviews = pd.read_csv('./CleanDatasets/reviews.csv')
df_games = pd.read_csv('./CleanDatasets/steam_games.csv')
df_items = pd.read_parquet('./CleanDatasets/itemsit.parquet')

# 01 Developer function

In [3]:
df_filtrado = df_games[['price', 'release_year', 'developer', 'item_id']]
# se eliminan los duplicados
df_dev = df_filtrado.drop_duplicates()
df_dev

,price,release_year,developer,item_id
0,4.99,2018,Kotoshiro,761140
5,0.00,2018,Secret Level SRL,643980
9,0.00,2017,Poolians.com,670290
14,0.99,2017,彼岸领域,767400
17,3.99,2018,Trickjump Games Ltd,772540
...,...,...,...,...
71538,1.99,2018,Bidoniera Games,745400
71542,1.99,2018,"Nikita ""Ghost_RUS""",773640
71546,4.99,2018,Sacada,733530
71549,1.99,2018,Laush Dmitriy Sergeevich,610660


In [4]:
# Convert the DataFrame to a PyArrow Table object
table = pa.Table.from_pandas(df_dev)

# Define the compression options with Snappy
compression = 'snappy'

# Save the Table object in Parquet format with Snappy compression
pq.write_table(table, r"C:\My Project\Integ\MLInt\Datasets\CleanDatasets\df_funct_dev.parquet", compression=compression)

# 02     User_Data function

df_items ===>  **df_expenses_items**

In [18]:
# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
df_expenses_items = df_items[['items_count', 'user_id', 'item_id']]
df_expenses_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094100,7,76561198329548331,346330
5094101,7,76561198329548331,373330
5094102,7,76561198329548331,388490
5094103,7,76561198329548331,521570


In [19]:
price_games = df_games[['price', 'item_id']]
# Remove duplicates
price_games = price_games.drop_duplicates(subset='item_id', keep='first')
price_games

,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71538,1.99,745400
71542,1.99,773640
71546,4.99,733530
71549,1.99,610660


In [20]:
price_games['item_id'] = price_games['item_id'].astype('int64')
df_expenses_items['item_id'] = df_expenses_items['item_id'].astype('int64')

df_expenses_items = df_expenses_items.merge(price_games, on='item_id', how='left')
df_expenses_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,NaN
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


In [21]:
# Convert the DataFrame to a PyArrow Table object
table = pa.Table.from_pandas(df_expenses_items)

# Define the compression options with Snappy
compression = 'snappy'

# Save the Table object in Parquet format with Snappy compression
pq.write_table(table, r"C:\My Project\Integ\MLInt\Datasets\CleanDatasets\df_expenses_items.parquet", compression=compression)

# 03     userforgenre function

playtime_forever_user to **df_playtime**

In [16]:
playtime_forever_user = df_items[['playtime_forever', 'user_id', 'item_id']]
playtime_forever_user

,playtime_forever,user_id,item_id
0,6.0,76561197970982479,10
1,0.0,76561197970982479,20
2,7.0,76561197970982479,30
3,0.0,76561197970982479,40
4,0.0,76561197970982479,50
...,...,...,...
5094100,0.0,76561198329548331,346330
5094101,0.0,76561198329548331,373330
5094102,3.0,76561198329548331,388490
5094103,4.0,76561198329548331,521570


In [26]:
genre_item = df_games[['genres', 'item_id','release_year']]
genre_item

,genres,item_id,release_year
0,Action,761140,2018
1,Casual,761140,2018
2,Indie,761140,2018
3,Simulation,761140,2018
4,Strategy,761140,2018
...,...,...,...
71549,Indie,610660,2018
71550,Racing,610660,2018
71551,Simulation,610660,2018
71552,Casual,658870,2017


In [27]:
# Convert 'item_id' column in both DataFrames to int
playtime_forever_user['item_id'] = playtime_forever_user['item_id'].astype(int)
genre_item['item_id'] = genre_item['item_id'].astype(int)

# Now you can merge
df_playtime_genre = playtime_forever_user.merge(genre_item, on='item_id')
df_playtime_genre

,playtime_forever,user_id,item_id,genres,release_year
0,6.0,76561197970982479,10,Action,2000
1,0.0,js41637,10,Action,2000
2,0.0,Riot-Punch,10,Action,2000
3,93.0,doctr,10,Action,2000
4,108.0,corrupted_soul,10,Action,2000
...,...,...,...,...,...
9877299,164.0,76561198107283457,354280,Indie,2016
9877300,164.0,76561198107283457,354280,Simulation,2016
9877301,0.0,inven,433920,Adventure,2016
9877302,0.0,inven,433920,Indie,2016


In [33]:
df_playtime_genre.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9877304 entries, 0 to 9877303
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   playtime_forever  float64
 1   user_id           object 
 2   item_id           int32  
 3   genres            object 
 4   release_year      object 
dtypes: float64(1), int32(1), object(3)
memory usage: 339.1+ MB


In [35]:
# Convert the DataFrame to a PyArrow Table object
table = pa.Table.from_pandas(df_playtime_genre)

# Define the compression options with Snappy
compression = 'snappy'

# Save the Table object in Parquet format with Snappy compression
pq.write_table(table, './CleanDatasets/df_playtime.parquet', compression=compression)

In [30]:
# Group by user and sum the playtime
agg_genre = df_playtime_genre.groupby(['release_year', 'user_id','genres'])['playtime_forever'].sum().reset_index()
# Convert the 'playtime_forever' column to hours
agg_genre['playtime_hours'] = agg_genre['playtime_forever']/60
# Drop the column in minutes
agg_genre = agg_genre.drop('playtime_forever', axis=1)
agg_genre


,release_year,user_id,genres,playtime_hours
0,1983,2Ta4,Action,0.300000
1,1983,2Ta4,Adventure,0.300000
2,1983,2Ta4,Casual,0.300000
3,1983,76561197960324641,Action,0.000000
4,1983,76561197960324641,Adventure,0.000000
...,...,...,...,...
3495277,Date no available,zzoptimuszz,Free to Play,46.883333
3495278,Date no available,zzoptimuszz,Massively Multiplayer,0.816667
3495279,Date no available,zzoptimuszz,RPG,0.816667
3495280,Date no available,zzyfo,Action,0.000000


playtime_forever_user to **df_genre_ranking**

In [11]:
# Group by 'genres' and sum 'playtime_hours'
df_genre_ranking = agg_genre.groupby('genres')['playtime_hours'].sum().reset_index()
# Sort by 'playtime_hours'
df_genre_ranking = df_genre_ranking.sort_values(by='playtime_hours', ascending=False)
# Add a column with the ranking position
df_genre_ranking['ranking'] = df_genre_ranking['playtime_hours'].rank(ascending=False).astype(int)
df_genre_ranking


,genres,playtime_hours,ranking
0,Action,5.125167e+07,1
9,Indie,2.459123e+07,2
12,RPG,1.713213e+07,3
1,Adventure,1.497939e+07,4
14,Simulation,1.425438e+07,5
17,Strategy,1.084995e+07,6
8,Free to Play,1.006072e+07,7
10,Massively Multiplayer,7.352093e+06,8
4,Casual,4.155277e+06,9
6,Early Access,2.611542e+06,10


In [12]:
# Convert the DataFrame to a PyArrow Table object
table = pa.Table.from_pandas(df_genre_ranking)

# Define the compression options with Snappy
compression = 'snappy'

# Save the Table object in Parquet format with Snappy compression
pq.write_table(table, r"C:\My Project\Integ\MLInt\Datasets\CleanDatasets\df_genre_ranking.parquet", compression=compression)

# 04 best_developer_year and 05 dev_reviews_analysis

release_year in to **df_reviews_L**

In [14]:
release_year_item = df_games[['item_id', 'release_year','developer']]
# Remove duplicates
release_year_item = release_year_item.drop_duplicates()
release_year_item

,item_id,release_year,developer
0,761140,2018,Kotoshiro
5,643980,2018,Secret Level SRL
9,670290,2017,Poolians.com
14,767400,2017,彼岸领域
17,772540,2018,Trickjump Games Ltd
...,...,...,...
71538,745400,2018,Bidoniera Games
71542,773640,2018,"Nikita ""Ghost_RUS"""
71546,733530,2018,Sacada
71549,610660,2018,Laush Dmitriy Sergeevich


In [15]:
print(df_reviews.columns)
print(release_year_item.columns)

Index(['user_id', 'user_url', 'sentiment_analysis', 'posted', 'item_id',
       'recommend'],
      dtype='object')
Index(['item_id', 'release_year', 'developer'], dtype='object')


In [16]:
df_reviews_l = df_reviews.merge(release_year_item, on='item_id')
df_reviews_l

,user_id,user_url,sentiment_analysis,posted,item_id,recommend,release_year,developer
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-11-05,1250,True,2009,Tripwire Interactive
1,death-hunter,http://steamcommunity.com/id/death-hunter,1,2015-03-30,1250,True,2009,Tripwire Interactive
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,1,2013-07-12,1250,True,2009,Tripwire Interactive
3,diego9031,http://steamcommunity.com/id/diego9031,1,2015-08-13,1250,True,2009,Tripwire Interactive
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,2,2014-04-05,1250,True,2009,Tripwire Interactive
...,...,...,...,...,...,...,...,...
49709,llDracuwulf,http://steamcommunity.com/id/llDracuwulf,1,2015-10-29,307130,True,2014,Legend Studio
49710,ChrisCoroner,http://steamcommunity.com/id/ChrisCoroner,1,2024-05-05,209120,True,2012,"Capcom U.S.A., Inc."
49711,MeloncraftLP,http://steamcommunity.com/id/MeloncraftLP,0,2024-08-13,220090,True,2013,SkyGoblin
49712,MeloncraftLP,http://steamcommunity.com/id/MeloncraftLP,0,2024-08-13,262850,True,2014,SkyGoblin


In [27]:
df_reviews_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49714 entries, 0 to 49713
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49714 non-null  object
 1   user_url            49714 non-null  object
 2   sentiment_analysis  49714 non-null  int64 
 3   posted              49714 non-null  object
 4   item_id             49714 non-null  int64 
 5   recommend           49714 non-null  bool  
 6   release_year        49714 non-null  object
 7   developer           49714 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 2.7+ MB


In [17]:
# Convert the DataFrame to a PyArrow Table object
table = pa.Table.from_pandas(df_reviews_l)

# Define the compression options with Snappy
compression = 'snappy'

# Save the Table object in Parquet format with Snappy compression
pq.write_table(table, r"C:\My Project\Integ\MLInt\Datasets\CleanDatasets\df_reviews_l.parquet", compression=compression)

In [32]:
df_reviews.columns

Index(['user_id', 'user_url', 'sentiment_analysis', 'posted', 'item_id',
       'recommend'],
      dtype='object')

In [22]:
def game_genre(genre):
    # Search for the ranking for the genre of interest
    rank = df_genre_ranking[df_genre_ranking['genres'] == genre]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [26]:
genre = 'Action'
game_genre(genre)

{'rank': 1}

Game Recommendation - id_product

In [ ]:
release_year_item = df_games[['item_id', 'release_year','developer']]
# Remove duplicates
release_year_item = release_year_item.drop_duplicates()
release_year_item